### DATA PT12 - Proyecto final - Grupo 2
Una notebook para todo. </br>
##### Transformaciones
gm_sitios:</br>
- columna 'address' queda en columnas separadas: domicilio como numero y calle, ciudad, esatdo y zip code </br>
- Columna 'hours' la separa en columnas (Monday, Tuesday ... Sunday) con el horario de cada dia
- Columna 'price' reemplazar '₩', '₩₩' por $ o $$ - esta forma de hacerlo debe ser muy lenta
- Columna 'MISC' como variables dummies. De la columna:</br>
    - Service options >> 'Delivery', 'Takeout', 'Dine-in'</br>
    - Amenities >> kids </br>
    - Atmosphere >> casual </br>
    - Popular for >> 'Lunch', 'Dinner'</br>

gm_reviews:
- columna 'time' esta en ms. La convierte a datetime para obtener fecha de la review

In [1]:
# import glob
# import matplotlib.pyplot as plt
import numpy as np
# import os
import pandas as pd
# import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
# leo el archivo parquet datos crudos
# How to read a Parquet file into Pandas DataFrame?
gm_sitios = pd.read_parquet("datos/gm_raw_parquet/gm_sitios_raw.parquet" , engine='fastparquet')

In [3]:
# Eliminamos columnas que no serán utilizadas
# NO ELIMINO 'price', 'MISC', PUEDE SERVIR PARA LAS RECOMENDACIONES
# Por ahora conservo latitude y longitude porque puede servirnos para mostrar locales en el dashboard

gm_sitios = gm_sitios.drop(columns=['description', 'state', 'relative_results', 'url'])

In [4]:
# selecciono las filas que tienen pizza restaurant
# df_mt_sitios_pizza = df_mt_sitios[df_mt_sitios['category'].apply(lambda x: 'Pizza restaurant' in x)]
# TypeError: argument of type 'NoneType' is not iterable

gm_sitios_pizza = gm_sitios[gm_sitios['category'].apply(lambda x: isinstance(x, list) and 'Pizza restaurant' in x)]

In [5]:
# Extract state (two uppercase letters) using regex
gm_sitios_pizza['state'] = gm_sitios_pizza['address'].str.extract(r',\s*([A-Z]{2})\s*\d{5}') 

In [6]:
# selecciono las pizzerias de NJ y NY
gm_sitios_pizza_NJNY = gm_sitios_pizza[gm_sitios_pizza['state'].isin(['NJ', 'NY'])]

In [7]:
# en algunas direcciones dice ', United States' al final / Remove ', United States' if present
gm_sitios_pizza_NJNY['cleaned_address'] = [addr.replace(", United States", "") for addr in gm_sitios_pizza_NJNY['address']]

In [8]:
# Regex pattern to extract street address, city, and ZIP code
pattern_1 = r'(?P<street_address_temp>.+),\s*(?P<city>[^,]+),\s*[A-Z]{2}\s*(?P<zip_code>\d{5})$'

# Aplicar regex a la columna 'address'
df_extracted = gm_sitios_pizza_NJNY['cleaned_address'].str.extract(pattern_1)

# Combinar con el DataFrame original
gm_sitios_pizza_NJNY = gm_sitios_pizza_NJNY.join(df_extracted)

In [9]:
# To capture only the part to the right of the comma
gm_sitios_pizza_NJNY["street_address"] = gm_sitios_pizza_NJNY["street_address_temp"].str.split(",", n=1).str[1].str.strip()

In [11]:
# elimino las columnas que ya no usamos
# df.drop(['Column1', 'Columns2'], axis=1)
gm_sitios_pizza_NJNY.drop(['address', 'cleaned_address', 'street_address_temp'], axis=1, inplace=True)

In [12]:
# elimino los duplicados manteniendo la primera instancia
gm_sitios_pizza_NJNY.drop_duplicates(subset=['gmap_id'], keep='first', inplace=True)

In [13]:
# Convert to dictionary format para extraer los horarios por dia de cada local. La columna es una lista de listas
gm_sitios_pizza_NJNY['hours_dict'] = gm_sitios_pizza_NJNY['hours'].apply(lambda x: dict(x) if isinstance(x, list) else {})

In [14]:
# Expand 'hours_dict' into separate columns (one per day)
hours_expanded = gm_sitios_pizza_NJNY['hours_dict'].apply(pd.Series)

In [15]:
# Merge expanded columns back into original DataFrame
gm_sitios_pizza_NJNY = pd.concat([gm_sitios_pizza_NJNY, hours_expanded], axis=1)

In [16]:
# reemplazar '₩', '₩₩' por $ o $$ - esta forma de hacerlo debe ser muy lenta
#replacements = str.maketrans({"h": "H", "e": "E", "o": "O"})
#res = s.translate(replacements)
# string keys in translate table must be of length 1
#gm_sitios_pizza_NJNY['price'] = (
#    gm_sitios_pizza_NJNY['price']
#    .str.replace("₩₩", "$$", regex=False)  # Primero reemplaza ₩₩ por $$
#    .str.replace("₩", "$", regex=False)    # Luego reemplaza ₩ por $
#)
gm_sitios_pizza_NJNY['price'] = gm_sitios_pizza_NJNY['price'].replace({'₩': '$', '₩₩': '$$'})

In [17]:
# completo con un diccionario vacios las filas que tienen la columna MISC en NaN
gm_sitios_pizza_NJNY['MISC'] = gm_sitios_pizza_NJNY['MISC'].apply(lambda x: {} if pd.isnull(x) else x)

In [18]:
# Expand 'MISC' into separate columns (one per iem)
MISC_expanded = gm_sitios_pizza_NJNY['MISC'].apply(pd.Series)

In [20]:
# elimino aca las columnas que no usare
MISC_expanded.drop(['Offerings', 'Dining options', 'Health & safety', 'Accessibility', 'Crowd', 'Planning', 
'Payments', 'Highlights', 'From the business', 'Health and safety'], axis=1, inplace=True)

In [21]:
# Merge expanded columns back into original DataFrame
# creo que se concatena por los indices. Por eso no necesita que haya una columna en comun
gm_sitios_pizza_NJNY = pd.concat([gm_sitios_pizza_NJNY, MISC_expanded], axis=1)

In [22]:
# df.drop(['Column1', 'Columns2'], axis=1)
# elimino columnas que ya no necesito
gm_sitios_pizza_NJNY.drop(['category', 'MISC', 'hours', 'hours_dict'], axis=1, inplace=True)

In [23]:
# COLUMNAS DUMMIES

# Apply function to filter only "Lunch" and "Dinner" and join them as a string
# Handle NaN values and apply filtering
#gm_sitios_pizza_NJNY['Service options'] = gm_sitios_pizza_NJNY['Service options'].apply(
#    lambda x: ', '.join([item for item in x if item in ['Delivery', 'Takeout', 'Dine-in']]) 
#    if isinstance(x, list) else ''
#)
gm_sitios_pizza_NJNY['Service options'] = gm_sitios_pizza_NJNY['Service options'].apply(
    lambda x: ','.join([item for item in x if item in ['Delivery', 'Takeout', 'Dine-in']]) 
    if isinstance(x, list) else ''
)

In [24]:
# Apply function to filter only "good for kids" and join them as a string
# Handle NaN values and apply filtering
gm_sitios_pizza_NJNY['Amenities'] = gm_sitios_pizza_NJNY['Amenities'].apply(
    lambda x: 'Good for kids' if isinstance(x, list) and 'Good for kids' in x else ''
)

In [25]:
# Apply function to Atmosphere to filter only "casual" and join them as a string
# Handle NaN values and apply filtering
gm_sitios_pizza_NJNY['Atmosphere'] = gm_sitios_pizza_NJNY['Atmosphere'].apply(
    lambda x: 'Casual' if isinstance(x, list) and 'Casual' in x else ''
)

In [26]:
# Apply function to filter only "Lunch" and "Dinner" and join them as a string
# Handle NaN values and apply filtering
gm_sitios_pizza_NJNY['Popular for'] = gm_sitios_pizza_NJNY['Popular for'].apply(
    lambda x: ','.join([item for item in x if item in ['Lunch', 'Dinner']]) 
    if isinstance(x, list) else ''
)

In [27]:
# Aplicar get_dummies() para convertir en variables dummies
df_dummies_serv = gm_sitios_pizza_NJNY['Service options'].str.get_dummies(sep=',')
df_dummies_am = gm_sitios_pizza_NJNY['Amenities'].str.get_dummies(sep=',')
df_dummies_at = gm_sitios_pizza_NJNY['Atmosphere'].str.get_dummies(sep=',')
df_dummies_pop = gm_sitios_pizza_NJNY['Popular for'].str.get_dummies(sep=',')

In [28]:
# df.drop(['Column1', 'Columns2'], axis=1)
# elimino columnas que ya no necesito
gm_sitios_pizza_NJNY.drop(['Service options', 'Amenities', 'Atmosphere', 'Popular for'], axis=1, inplace=True)

In [29]:
# Concatenate all DataFrames along columns
gm_sitios_pizza_NJNY = pd.concat([gm_sitios_pizza_NJNY, df_dummies_serv, df_dummies_am, df_dummies_at, df_dummies_pop], axis=1)

In [30]:
# Leo los datasets de reviews completos
df_rev_NJ = pd.read_parquet('datos/gm_raw_parquet/gm_review_NJ_raw.parquet' , engine='fastparquet')
df_rev_NY = pd.read_parquet('datos/gm_raw_parquet/gm_review_NY_raw.parquet' , engine='fastparquet')

In [31]:
# Eliminamos, por ahora, name, pics y resp. no usaremos esas columnas
df_rev_NJ = df_rev_NJ.drop(columns=['name', 'pics', 'resp'])
df_rev_NY = df_rev_NY.drop(columns=['name', 'pics', 'resp'])

In [32]:
# eliminar los duplicados manteniendo la primera instancia
df_rev_NJ = df_rev_NJ.drop_duplicates(subset=['user_id', 'gmap_id', 'time'], keep='first')
df_rev_NY = df_rev_NY.drop_duplicates(subset=['user_id', 'gmap_id', 'time'], keep='first')

In [33]:
# convertir columna time en int64 a datetime
df_rev_NJ['date'] = pd.to_datetime(df_rev_NJ['time'], unit='ms')
df_rev_NY['date'] = pd.to_datetime(df_rev_NY['time'], unit='ms')

# Eliminamos la columan time
df_rev_NJ = df_rev_NJ.drop(columns=['time'])
df_rev_NY = df_rev_NY.drop(columns=['time'])

In [34]:
# uno los dataframes de reviews hacer join con el dataframe de sitios /
# asi solo conservare los locales para los que haya reviews y las reviews para las que conozcamos los locales
df_rev_NJNY = pd.concat([df_rev_NJ, df_rev_NY], ignore_index=True)

In [39]:
# merge entre los dos dataframe . Luego guardare solo los que tengan datos en ambos.
gm_NJNY = pd.merge(gm_sitios_pizza_NJNY, df_rev_NJNY, how='inner', on='gmap_id') # tengo que hacer inner join

In [ ]:
# vuelvo a separar los archivos para dejarlos listos para la base de datos
# unique_age_city = df[['Age', 'City']].drop_duplicates()
gm_sitios_NJNY = gm_NJNY[['gmap_id', 'name', 'street_address', 'city', 'state', 'zip_code',   'latitude','longitude','avg_rating','num_of_reviews','price',
                   'Monday','Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'Delivery', 'Dine-in', 'Takeout', 'Good for kids', 'Casual', 
                   'Dinner', 'Lunch']].drop_duplicates()

In [45]:
# ahora selecciono las reviews
gm_rev_NJNY = gm_NJNY[['gmap_id', 'user_id', 'date', 'rating' , 'text']]

Dataframes listos para cargar en las tablas de la base de datos </br>
La siguiente celda, donde grabo los archivos, no es necesaria

In [46]:
# guardo gm_sitios_NJNY
gm_sitios_NJNY.to_parquet('datos/gm_raw_parquet/gm_sitios_NJNY_202502190828.parquet' , engine='fastparquet')

# guardo gm_rev_NJNY
#gm_rev_NJNY.to_parquet('datos/gm_raw_parquet/gm_rev_NJNY_202502182030.parquet' , engine='fastparquet')